## 1. 필수 라이브러리 설치
##### LangChain과 벡터 DB Chroma, 임베딩 모델을 위한 라이브러리를 설치
##### pip install langchain langchain-community langchain-huggingface chromadb sentence-transformers

##### PDF를 읽기 위해 pypdf 라이브러리 추가
##### pip install pypdf

## 2. RAG 구현 코드
##### 특정 텍스트 문서(예: 상식 데이터)를 기반으로 TinyLlama가 답변

In [1]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json
import os

from config import config

# 1. 모델 및 임베딩 설정 TinyLlama 모델 로드 (Fine-tuning한 모델 경로)
model_id = config.save_path
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=256, 
    temperature=0.1,
)
llm = HuggingFacePipeline(pipeline=pipe)
# # TODO HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기
embeddings = HuggingFaceEmbeddings(model_name=config.embedding_path,
                                   model_kwargs={'device': 'cuda',
                                                 # # `weights_only=True` error 시, 내부 Transformer 모델에 전달 (보안때문에 추천하지 않음, 서버 성능 좋으면 torch 2.6 이상으로 업글 요망) 
                                                'model_kwargs': {'weights_only': False}, 
                                                },
                                )

# 2. 문서 및 벡터DB 설정 (PDF 및 텍스트 파일 통합 로드)
all_documents = []

# # 2-1. 기존 JSONL 데이터 로드
if os.path.exists(config.jsonl_train_data_path):
    with open(config.jsonl_train_data_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            combined_text = f"질문: {data['instruction']}\n답변: {data['output']}"
            all_documents.append(Document(page_content=combined_text, metadata={"source": "jsonl"}))

# # 2-2. PDF 파일 로드
pdf_loader = DirectoryLoader(config.domain_data_dir, glob="**/*.pdf", loader_cls=PyPDFLoader)
all_documents.extend(pdf_loader.load())

# # 2-3. 텍스트 파일 로드
txt_loader = DirectoryLoader(config.domain_data_dir, glob="**/*.txt", loader_cls=TextLoader)
all_documents.extend(txt_loader.load())

# # 2-4. 문서 분할 (Chunking)
# 문장이 긴 경우, 적절한 분할이 필수
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,    # 도메인 지식의 맥락을 위해 조금 더 크게 설정 가능
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "] # 분할 기준 우선순위
)
final_texts = text_splitter.split_documents(all_documents)

# # 2-5. 벡터 DB 생성
vectorstore = Chroma.from_documents(
    documents=final_texts, 
    embedding=embeddings,
    # persist_directory="./input/chroma_db"  # DB를 로컬에 저장하고 싶을 경우 지정
)

# # 2-6. retriever 설정
# k값 5에서 3으로 줄여 프롬프트에 들어가는 텍스트 양을 제한
# TinyLlama는 문맥 파악 능력 크지 않아, 너무 큰 정보 주면 답변 못 함... 하...
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 3. [중요] LCEL 방식의 Chain 구성 (RetrievalQA 대체)
template = """참고 문맥을 바탕으로 질문에 답하세요:
{context}

질문: {question}
답변:"""

prompt = ChatPromptTemplate.from_template(template)

# LCEL 파이프라인 구성: 질문 -> 컨텍스트 검색 -> 프롬프트 생성 -> 모델 실행 -> 결과 출력
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행
response = rag_chain.invoke("대한민국의 수도는 어디야?")
print(response)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Human: 참고 문맥을 바탕으로 질문에 답하세요:
[Document(metadata={'total_pages': 40, 'source': 'input/domain_data/개인정보 보호법(법률)(제20897호)(20251002).pdf', 'page_label': '17', 'creationdate': '2026-01-09T11:17:18+09:00', 'creator': 'PyPDF', 'moddate': '2026-01-09T11:17:18+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'page': 16}, page_content='해당 국가의 수준에 상응하는 제한을 할 수 있다. 다만, 조약 또는 그 밖의 국제협정의 이행에 필요한 경우에는 그러\n하지 아니하다.\n[본조신설 2023. 3. 14.]\n \n제28조의11(준용규정) 제28조의8제1항 각 호 외의 부분 단서에 따라 개인정보를 이전받은 자가 해당 개인정보를 제\n3국으로 이전하는 경우에 관하여는 제28조의8 및 제28조의9를 준용한다. 이 경우 “개인정보처리자”는 “개인정보를\n이전받은 자”로, “개인정보를 이전받는 자”는 “제3국에서 개인정보를 이전받는 자”로 본다.\n[본조신설 2023. 3. 14.]'), Document(metadata={'source': 'jsonl'}, page_content="질문: 세종대왕의 업적 중 가장 대표적인 것은?\n답변: 가장 대표적인 업적은 백성들이 쉽게 글을 익힐 수 있도록 '훈민정음(한글)'을 창제하신 것입니다."), Document(metadata={'creationdate': '2026-01-09T11:17:18+09:00', 'moddate': '2026-01-09T11:17:18+09:00', 'page_label': '13', 'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 40, 'creator': 'PyPDF', 'source': 

In [ ]:
for rqst in ["14세 미만 아동의 개인정보 처리를 위해 누구의 동의를 받아야 하는가?", "서버 기본 포트는?", "과일 중 사과는 무슨색인가요?", "하늘을 좋아하나요?", "하늘은 무슨색인가요?", "하늘은 왜 파란가요?", "노벨상을 만든 사람은 누구인가요?", "목감기에 좋은 차를 추천해줘.", "'잘 자.'를 영어로.", "이진수 1010을 십진수로 바꾸면?"]:
    response = rag_chain.invoke(rqst)
    print(f"res:::::::::::::::::::::\n{response}\n\n")

res:::::::::::::::::::::
Human: 참고 문맥을 바탕으로 질문에 답하세요:
[Document(metadata={'creationdate': '2026-01-09T11:17:18+09:00', 'page': 10, 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'source': 'input/domain_data/개인정보 보호법(법률)(제20897호)(20251002).pdf', 'moddate': '2026-01-09T11:17:18+09:00', 'total_pages': 40, 'page_label': '11'}, page_content='② 제1항에도 불구하고 법정대리인의 동의를 받기 위하여 필요한 최소한의 정보로서 대통령령으로 정하는 정보는\n법정대리인의 동의 없이 해당 아동으로부터 직접 수집할 수 있다.\n③ 개인정보처리자는 만 14세 미만의 아동에게 개인정보 처리와 관련한 사항의 고지 등을 할 때에는 이해하기 쉬운\n양식과 명확하고 알기 쉬운 언어를 사용하여야 한다.\n④ 제1항부터 제3항까지에서 규정한 사항 외에 동의 및 동의 확인 방법 등에 필요한 사항은 대통령령으로 정한다.\n[본조신설 2023. 3. 14.]\n \n         제2절 개인정보의 처리 제한'), Document(metadata={'page': 36, 'page_label': '37', 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'moddate': '2026-01-09T11:17:18+09:00', 'source': 'input/domain_data/개인정보 보호법(법률)(제20897호)(20251002).pdf', 'total_pages': 40, 'creationdate': '2026-01-09T11:17:18+09:00'}, page_content='법제처                

## 3. 최적화 포인트
##### 1) Embedding Model (검색의 핵심)
###### HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기 (e.g ko-sroberta-multitask)

##### 2) Prompt Engineering (소형 모델의 가이드)
###### TinyLlama는 매개변수가 적기 때문에 프롬프트가 매우 중요 ### 참고 문맥:과 같이 명확한 구분자를 주어 모델이 어디를 읽고 어디에 대답해야 하는지 명시.

##### 3) Temperature 설정
###### RAG에서는 모델의 창의성보다 문서에 기반한 정확성이 중요. 따라서 temperature를 0.1에 가깝게 낮게 설정하여 멋대로 말을 지어내는 '할루시네이션'을 방지.

##### 4) 소형 모델의 한계 극복
###### TinyLlama가 문맥이 너무 길면 답변 어려움. chunk_size를 200~300 정도로 짧게 유지, k값(가져올 문서 개수)을 2~3개 정도로 제한(for 2080 Ti 메모리와 모델 성능 모두에 유리)